In [1]:
import tensorflow as tf

## Create Datasets
Tensorflow provides various methods to create Datasets from numpy arrays, text files, CSV files, tensors, etc.

In [2]:
# Source data - numpy array
data = np.arange(10)

# Create a dataset from numpy array
dataset = tf.data.Dataset.from_tensor_slices(data)

In [4]:
# Source data - tensor
data = tf.arange(10) # Tensorflow v 2.0

dataset = tf.data.Dataset.from_tensors(data)

AttributeError: module 'tensorflow' has no attribute 'arange'

In [5]:
# Source data - generators

def generator():
    for i in range(10):
        yield 2*i

dataset = tf.data.Dataset.from_generator(generator, (tf.int32))

## Operations on Datasets

In [7]:
# Batches

data = np.arange(10, 40)

# Create batches of 10
dataset = tf.data.Dataset.from_tensor_slices(data).batch(10)

# Create iterator to consume the data
iterator = dataset.make_one_shot_iterator()
next_ele = iterator.get_next()

with tf.Session() as sess:
    try: 
        while True:
            val = sess.run(next_ele)
            print(val)
    except tf.errors.OutOfRangeError:
        pass


[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]


In [9]:
# Zip 

datax = np.arange(10, 20)
datay = np.arange(11, 21)

datasetx = tf.data.Dataset.from_tensor_slices(datax)
datasety = tf.data.Dataset.from_tensor_slices(datay)

dcombined = tf.data.Dataset.zip((datasetx, datasety)).batch(2)
iterator = dcombined.make_one_shot_iterator()
next_ele = iterator.get_next()

with tf.Session() as sess:
    try:
        while True:
            val = sess.run(next_ele)
            print(val)
    except tf.errors.OutOfRangeError:
        pass

(array([10, 11]), array([11, 12]))
(array([12, 13]), array([13, 14]))
(array([14, 15]), array([15, 16]))
(array([16, 17]), array([17, 18]))
(array([18, 19]), array([19, 20]))


In [10]:
# Repeat

dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
dataset = dataset.repeat(count=2)
iterator = dataset.make_one_shot_iterator()
next_ele = iterator.get_next()

with tf.Session() as sess:
    try:
        while True:
            val = sess.run(next_ele)
            print(val)
    except tf.errors.OutOfRangeError:
        pass

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


In [12]:
# Map: Transfor data before feeding into the model

def map_fnc(x):
    return x*2;

data = np.arange(10)
dataset = tf.data.Dataset.from_tensor_slices(data)
dataset = dataset.map(map_fnc)

iterator = dataset.make_one_shot_iterator()
next_ele = iterator.get_next()

with tf.Session() as sess:
    try:
        while True:
            val = sess.run(next_ele)
            print(val)
    except tf.errors.OutOfRangeError:
        pass

0
2
4
6
8
10
12
14
16
18


## Creating Iterators

Tensorflow provides Iterators to consume the data

In [14]:
# one-shot iterator

data = np.arange(10, 15)
dataset = tf.data.Dataset.from_tensor_slices(data)

# create iterator 
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()
with tf.Session() as sess:
    val = sess.run(next_element)
    print(val)

10


In [15]:
# initializable iterator

# define two placeholders to accept min and max value
min_val = tf.placeholder(tf.int32, shape=[])
max_val = tf.placeholder(tf.int32, shape=[])

data = tf.range(min_val, max_val)
dataset = tf.data.Dataset.from_tensor_slices(data)

iterator = dataset.make_initializable_iterator()
next_ele = iterator.get_next()
with tf.Session() as sess:
    
    # Initialize an iterator with range of values from 10 to 15
    sess.run(iterator.initializer, feed_dict={min_val:10, max_val:15})
    try:
        while True:
            val = sess.run(next_ele)
            print(val)
    except tf.errors.OutOfRangeError:
        pass
    
    # initialize an iterator with range of values from 1 to 10
    sess.run(iterator.initializer, feed_dict={min_val:1, max_val:10})
    try:
        while True:
            val = sess.run(next_ele)
            print(val)
    except tf.errors.OutOfRangeError:
        pass

10
11
12
13
14
1
2
3
4
5
6
7
8
9


In [16]:
# Reinitializable iterator 

def map_fnc(ele):
    return ele*2

min_val = tf.placeholder(tf.int32, shape=[])
max_val = tf.placeholder(tf.int32, shape=[])
data = tf.range(min_val, max_val)

# Define separate datasets fro training and validation
train_dataset = tf.data.Dataset.from_tensor_slices(data)
val_dataset = tf.data.Dataset.from_tensor_slices(data).map(map_fnc)

# Create iterator 
iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)

train_initializer = iterator.make_initializer(train_dataset)
val_initializer = iterator.make_initializer(val_dataset)

next_ele = iterator.get_next()
with tf.Session() as sess:
    
    # initialize an iterator with range of values from 10 to 15
    sess.run(train_initializer, feed_dict={min_val:10, max_val:15})
    try:
        while True:
            val = sess.run(next_ele)
            print(val)
    except tf.errors.OutOfRangeError:
        pass
    
    # initialize an iterator with range of values from 1 to 10
    sess.run(train_initializer, feed_dict={min_val:1, max_val:10})
    try:
        while True:
            val = sess.run(next_ele)
            print(val)
    except tf.errors.OutOfRangeError:
        pass

10
11
12
13
14
1
2
3
4
5
6
7
8
9


In [17]:
# Feedable iterator: Can be used to switch between Iterators for different Datasets.

def map_fnc(ele):
    return ele*2

min_val = tf.placeholder(tf.int32, shape=[])
max_val = tf.placeholder(tf.int32, shape=[])
data = tf.range(min_val, max_val)

# Define separate datasets fro training and validation
train_dataset = tf.data.Dataset.from_tensor_slices(data)
val_dataset = tf.data.Dataset.from_tensor_slices(data).map(map_fnc)

train_val_iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
train_initializer = train_val_iterator.make_initializer(train_dataset)
val_initializer = train_val_iterator.make_initializer(val_dataset)

test_dataset = tf.data.Dataset.from_tensor_slices(tf.range(10, 15))
test_iterator = test_dataset.make_one_shot_iterator()

handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle, train_dataset.output_types, train_dataset.output_shapes)
next_ele = iterator.get_next()

with tf.Session() as sess:
    
    train_val_handle = sess.run(train_val_iterator.string_handle())
    test_handle = sess.run(test_iterator.string_handle())
    
    # training
    sess.run(train_initializer, feed_dict={min_val:10, max_val:15})
    try:
        while True:
            val = sess.run(next_ele, feed_dict={handle:train_val_handle})
            print(val)
    except tf.errors.OutOfRangeError:
        pass
    
    # validation
    sess.run(train_initializer, feed_dict={min_val:1, max_val:10})
    try:
        while True:
            val = sess.run(next_ele, feed_dict={handle:train_val_handle})
            print(val)
    except tf.errors.OutOfRangeError:
        pass
    
    # testing
    try:
        while True:
            val = sess.run(next_ele, feed_dict={handle:test_handle})
            print(val)
    except tf.errors.OutOfRangeError:
        pass

10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
12
13
14


## LeNet-5 Model 

In [19]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", reshape=False, one_hot=True)

X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels

X_train = np.pad(X_train, ((0,0), (2,2), (2,2), (0,0)), 'constant')
X_val = np.pad(X_val, ((0,0), (2,2), (2,2), (0,0)), 'constant')
X_test = np.pad(X_test, ((0,0), (2,2), (2,2), (0,0)), 'constant')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [23]:
def forward_pass(X):
    W1 = tf.get_variable("W1", [5,5,1,6], initializer = tf.contrib.layers.xavier_initializer(seed=0))
    # for conv layer2
    W2 = tf.get_variable("W2", [5,5,6,16], initializer = tf.contrib.layers.xavier_initializer(seed=0))
    Z1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding='VALID')
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1, ksize = [1,2,2,1], strides = [1,2,2,1], padding='VALID')
    Z2 = tf.nn.conv2d(P1, W2, strides = [1,1,1,1], padding='VALID')
    A2= tf.nn.relu(Z2)
    P2= tf.nn.max_pool(A2, ksize = [1,2,2,1], strides=[1,2,2,1], padding='VALID')
    P2 = tf.contrib.layers.flatten(P2)
   
    Z3 = tf.contrib.layers.fully_connected(P2, 120)
    Z4 = tf.contrib.layers.fully_connected(Z3, 84)
    Z5 = tf.contrib.layers.fully_connected(Z4,10, activation_fn= None)
    
    return Z5

def model(X,Y):
    
    logits = forward_pass(X)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0009)
    learner = optimizer.minimize(cost)
    correct_predictions = tf.equal(tf.argmax(logits,1),   tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    
    return (learner, accuracy)

In [24]:
# one-shot iterator

epochs = 10 
batch_size = 64 
iterations = len(y_train) * epochs

tf.reset_default_graph()
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

# need to repeat the dataset for epoch number of times, as all the data needs
# to be fed to the dataset at once

dataset = dataset.repeat(epochs).batch(batch_size)
iterator = dataset.make_one_shot_iterator()

X_batch , Y_batch = iterator.get_next()
(learner, accuracy) = model(X_batch, Y_batch)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
  
    total_accuracy = 0
try:
    while True:
        temp_accuracy, _ = sess.run([accuracy, learner])
        total_accuracy += temp_accuracy
except tf.errors.OutOfRangeError:
    pass
  
print('Avg training accuracy is {}'.format((total_accuracy * batch_size) / iterations ))

RuntimeError: Attempted to use a closed Session.

In [25]:
# initializable iterator

epochs = 10 
batch_size = 64
tf.reset_default_graph()

X_data = tf.placeholder(tf.float32, [None, 32,32,1])
Y_data = tf.placeholder(tf.float32, [None, 10])

dataset = tf.data.Dataset.from_tensor_slices((X_data, Y_data))
dataset = dataset.batch(batch_size)

iterator = dataset.make_initializable_iterator()
X_batch , Y_batch = iterator.get_next()
(learner, accuracy) = model(X_batch, Y_batch)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        # train the model
        sess.run(iterator.initializer, feed_dict={X_data:X_train, Y_data:y_train})
        total_train_accuracy = 0
        no_train_examples = len(y_train)
        try:
            while True:
                temp_train_accuracy, _ = sess.run([accuracy, learner])
                total_train_accuracy += temp_train_accuracy*batch_size
        except tf.errors.OutOfRangeError:
            pass

        # validate the model
        sess.run(iterator.initializer, feed_dict={X_data:X_val, Y_data:y_val})
        total_val_accuracy = 0
        no_val_examples = len(y_val)
        try:
            while True:
                temp_val_accuracy = sess.run(accuracy)
                total_val_accuracy += temp_val_accuracy*batch_size
        except tf.errors.OutOfRangeError:
            pass

        print('Epoch {}'.format(str(epoch+1)))
        print("---------------------------")
        print('Training accuracy is {}'.format(total_train_accuracy/no_train_examples))
        print('Validation accuracy is {}'.format(total_val_accuracy/no_val_examples))

Epoch 1
---------------------------
Training accuracy is 0.9212545454545454
Validation accuracy is 0.9724
Epoch 2
---------------------------
Training accuracy is 0.9742727272727273
Validation accuracy is 0.9856
Epoch 3
---------------------------
Training accuracy is 0.9822909090909091
Validation accuracy is 0.9926
Epoch 4
---------------------------
Training accuracy is 0.9867636363636364
Validation accuracy is 0.9952
Epoch 5
---------------------------
Training accuracy is 0.9900363636363636
Validation accuracy is 0.995
Epoch 6
---------------------------
Training accuracy is 0.9922181818181818
Validation accuracy is 0.995
Epoch 7
---------------------------
Training accuracy is 0.9938545454545454
Validation accuracy is 0.9956
Epoch 8
---------------------------
Training accuracy is 0.9940727272727272
Validation accuracy is 0.9974
Epoch 9
---------------------------
Training accuracy is 0.9952727272727273
Validation accuracy is 0.9984
Epoch 10
---------------------------
Training ac

In [27]:
# Re-initializable iterator
def map_fnc(X, Y):
     return X, Y
    
epochs = 10 
batch_size = 64
tf.reset_default_graph()
X_data = tf.placeholder(tf.float32, [None, 32,32,1])
Y_data = tf.placeholder(tf.float32, [None, 10])

train_dataset = tf.data.Dataset.from_tensor_slices((X_data, Y_data)).batch(batch_size).map(map_fnc)
val_dataset =  tf.data.Dataset.from_tensor_slices((X_data, Y_data)).batch(batch_size)

iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
X_batch , Y_batch = iterator.get_next()
(learner, accuracy) = model(X_batch, Y_batch)

train_initializer = iterator.make_initializer(train_dataset)
val_initializer =  iterator.make_initializer(val_dataset)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
    
        # train the model
        sess.run(train_initializer, feed_dict={X_data:X_train, Y_data:y_train})
        total_train_accuracy = 0
        no_train_examples = len(y_train)
        try:
            while True:
                temp_train_accuracy, _ = sess.run([accuracy, learner])
                total_train_accuracy += temp_train_accuracy*batch_size
        except tf.errors.OutOfRangeError:
            pass

        # validate the model
        sess.run(val_initializer, feed_dict={X_data:X_val, Y_data:y_val})
        total_val_accuracy = 0
        no_val_examples = len(y_val)
        try:
            while True:
                temp_val_accuracy = sess.run(accuracy)
                total_val_accuracy += temp_val_accuracy*batch_size
        except tf.errors.OutOfRangeError:
            pass

        print('Epoch {}'.format(str(epoch+1)))
        print("---------------------------")
        print('Training accuracy is {}'.format(total_train_accuracy/no_train_examples))
        print('Validation accuracy is {}'.format(total_val_accuracy/no_val_examples))

Epoch 1
---------------------------
Training accuracy is 0.9188363636363637
Validation accuracy is 0.9666
Epoch 2
---------------------------
Training accuracy is 0.974733333310214
Validation accuracy is 0.9856
Epoch 3
---------------------------
Training accuracy is 0.9822909090909091
Validation accuracy is 0.9952
Epoch 4
---------------------------
Training accuracy is 0.9864
Validation accuracy is 0.994
Epoch 5
---------------------------
Training accuracy is 0.9892727272727273
Validation accuracy is 0.9976
Epoch 6
---------------------------
Training accuracy is 0.992
Validation accuracy is 0.9918
Epoch 7
---------------------------
Training accuracy is 0.9927636363636364
Validation accuracy is 0.9982
Epoch 8
---------------------------
Training accuracy is 0.993890909090909
Validation accuracy is 0.9972
Epoch 9
---------------------------
Training accuracy is 0.9937454545454546
Validation accuracy is 0.9968
Epoch 10
---------------------------
Training accuracy is 0.99514545454545

In [32]:
# Feedable iterator

epochs = 10 
batch_size = 64
tf.reset_default_graph()

X_data = tf.placeholder(tf.float32, [None, 32,32,1])
Y_data = tf.placeholder(tf.float32, [None, 10])

train_dataset = tf.data.Dataset.from_tensor_slices((X_data, Y_data)).batch(batch_size)
val_dataset =  tf.data.Dataset.from_tensor_slices((X_data, Y_data)).batch(batch_size)
test_dataset =  tf.data.Dataset.from_tensor_slices((X_test, y_test.astype(np.float32))).batch(batch_size)

handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle, train_dataset.output_types, train_dataset.output_shapes)
X_batch , Y_batch = iterator.get_next()
(learner, accuracy) = model(X_batch, Y_batch)

train_val_iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
train_iterator = train_val_iterator.make_initializer(train_dataset)
val_iterator = train_val_iterator.make_initializer(val_dataset)
test_iterator = test_dataset.make_one_shot_iterator()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_val_string_handle = sess.run(train_val_iterator.string_handle())
    test_string_handle = sess.run(test_iterator.string_handle())

    for epoch in range(epochs):

        # train the model
        sess.run(train_iterator, feed_dict={X_data:X_train, Y_data:y_train})
        total_train_accuracy = 0
        no_train_examples = len(y_train)
        try:
            while True:
                temp_train_accuracy, _ = sess.run([accuracy, learner], feed_dict={handle:train_val_string_handle})
                total_train_accuracy += temp_train_accuracy*batch_size
        except tf.errors.OutOfRangeError:
            pass

        # validate the model
        sess.run(val_iterator, feed_dict={X_data:X_val, Y_data:y_val})
        total_val_accuracy = 0
        no_val_examples = len(y_val)
        try:
            while True:
                temp_val_accuracy, _ = sess.run([accuracy, learner], feed_dict={handle:train_val_string_handle})
                total_val_accuracy += temp_val_accuracy*batch_size
        except tf.errors.OutOfRangeError:
            pass

        print('Epoch {}'.format(str(epoch+1)))
        print("---------------------------")
        print('Training accuracy is {}'.format(total_train_accuracy/no_train_examples))
        print('Validation accuracy is {}'.format(total_val_accuracy/no_val_examples))


    print("Testing the model --------")

    total_test_accuracy = 0
    no_test_examples = len(y_test)
    try:
        while True:
            temp_test_accuracy, _ = sess.run([accuracy, learner], feed_dict={handle:test_string_handle})
            total_test_accuracy += temp_test_accuracy*batch_size
    except tf.errors.OutOfRangeError:
        pass

    print('Testing accuracy is {}'.format(total_test_accuracy/no_test_examples))

Epoch 1
---------------------------
Training accuracy is 0.9195272727272727
Validation accuracy is 0.9768
Epoch 2
---------------------------
Training accuracy is 0.9747090909090909
Validation accuracy is 0.991
Epoch 3
---------------------------
Training accuracy is 0.9828181818181818
Validation accuracy is 0.995
Epoch 4
---------------------------
Training accuracy is 0.9870181818181818
Validation accuracy is 0.9988
Epoch 5
---------------------------
Training accuracy is 0.9895090909090909
Validation accuracy is 1.0012
Epoch 6
---------------------------
Training accuracy is 0.9917636363636364
Validation accuracy is 1.0018
Epoch 7
---------------------------
Training accuracy is 0.9935272727272727
Validation accuracy is 1.0052
Epoch 8
---------------------------
Training accuracy is 0.9943090909090909
Validation accuracy is 1.0028
Epoch 9
---------------------------
Training accuracy is 0.9948
Validation accuracy is 1.0054
Epoch 10
---------------------------
Training accuracy is 0.